In [48]:
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path

In [49]:
df = pd.read_csv('C:/Users/norri/DataspellProjects/mercury-ds/attribution/TimeSeries/BQ_11_15_22/EO_campaign.csv')
df.head(5)

,week,sales
0,2017-10-01,303.73
1,2017-10-01,61.14
2,2017-10-01,119.61
3,2017-10-01,845.43
4,2017-10-01,489.22


The following three code blocks have to be adjusted for different datasets,
but they all standardize the columns for the end of the program.
There is also a drop NA if needed

In [50]:
# df.drop(['campaign_type_long', 'campaign_type', 'spend', 'impressions'], axis=1, inplace=True)
# df.head(5)

In [53]:
df.rename(columns={'week': 'Date', 'sales': 'Revenue'}, inplace=True)
df = df.dropna()

In [54]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df.head(5)

,Date,Revenue
0,2017-10-01,303.73
1,2017-10-01,61.14
2,2017-10-01,119.61
3,2017-10-01,845.43
4,2017-10-01,489.22


I felt the clearest way to aggregate to weekly was to number each week
and sort by column and week number. It gives a reliable way to groupby in the next section,

In [55]:
df['Week_Number'] = df['Date'].dt.isocalendar().week
df['Year'] = df['Date'].dt.isocalendar().year
df.head(5)

,Date,Revenue,Week_Number,Year
0,2017-10-01,303.73,39,2017
1,2017-10-01,61.14,39,2017
2,2017-10-01,119.61,39,2017
3,2017-10-01,845.43,39,2017
4,2017-10-01,489.22,39,2017


In [56]:
df2 = df.groupby(['Year', 'Week_Number']).sum()
df2.head(5)

Revenue
Year Week_Number         
2017 39           2042.01
     40           4042.41
     41           2094.59
     42           2315.04
     43           3513.06

The original dataframe still has every observation from each day, but df2 has a repeated revenue for each sum it did for each week

In [57]:
df = df.merge(df2, how='inner', left_on=['Year', 'Week_Number'], right_on=['Year', 'Week_Number'])
df.head(5)

,Date,Revenue_x,Week_Number,Year,Revenue_y
0,2017-10-01,303.73,39,2017,2042.01
1,2017-10-01,61.14,39,2017,2042.01
2,2017-10-01,119.61,39,2017,2042.01
3,2017-10-01,845.43,39,2017,2042.01
4,2017-10-01,489.22,39,2017,2042.01


After merging back together, we need to first drop all of the repeated Sales values, so whenever
a Year, Week_Number, and a Sales are a duplicate, all but the first are dropped.

In [58]:
df['Sales'] = df['Revenue_y']

In [59]:
df = df.drop_duplicates(subset=['Year', 'Week_Number', 'Sales'], keep='first')
df.head(5)

,Date,Revenue_x,Week_Number,Year,Revenue_y,Sales
0,2017-10-01,303.73,39,2017,2042.01,2042.01
6,2017-10-02,0.00,40,2017,4042.41,4042.41
41,2017-10-09,106.53,41,2017,2094.59,2094.59
69,2017-10-16,0.00,42,2017,2315.04,2315.04
96,2017-10-23,43.96,43,2017,3513.06,3513.06


The index is reset because it has gaps in it from all the dropped variables.

In [60]:
df = df.drop(['Revenue_x', 'Revenue_y'], axis=1)
df = df.iloc[1: , :]
df = df.reset_index(drop=True)

In [61]:
df = df.drop(['Week_Number', 'Year'], axis=1)

In [62]:
df.head(10)

,Date,Sales
0,2017-10-02,4042.41
1,2017-10-09,2094.59
2,2017-10-16,2315.04
3,2017-10-23,3513.06
4,2017-10-30,1863.94
5,2017-11-06,3332.38
6,2017-11-13,3198.49
7,2017-11-20,2922.20
8,2017-11-27,1885.98
9,2017-12-04,2161.47


Finally, two files are saved out: one with all but the last four in the file, and the other with only the last four in the file.

In [63]:
df_training = df.iloc[:-4,:]
df_training.to_csv('eo_training.csv', header=True, index=False)

In [64]:
df_test = df.iloc[-4:,:]
df_test.to_csv('eo_test.csv', header=True, index=False)